In [1]:
#Grid de la segunda parte, en la que utilizamos ya la primera parte bien
#Esta va a ser la de las 44 variables con combined


#importamos paquetes
import sys
sys.path.append('Src/')
from data_modificado import * #hay funciones que estan cambiadas en este script para adaptralas a nuestro dataset
from train_2 import * #este hubo que modificar una linea tambien
from transfer_learning import * #hubo que modificart lo mismo que en train_2
from test_functions import *
from layers import *
from utils import *
from loss import *
from metric import *
from results import *
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import itertools as it

#funciones
def read_df(
              metadata_names=['age','Temperature','Precipitation3Days'],
              random_state=42,
              otu_filename='../Datasets/otu_table_all_80.csv',
              metadata_filename='../Datasets/metadata_table_all_80.csv'):
    otu = pd.read_csv(otu_filename, index_col=0, header=None).T
    #print(otu.head())
    otu = otu.set_index('otuids')
    otu = otu.astype('int32')
    metadata = pd.read_csv(metadata_filename)
    #print(metadata.head())
    metadata = metadata.set_index('X.SampleID')
    metadata.head()
    domain = metadata[metadata_names]
    #if 'INBREDS' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['INBREDS'], prefix='INBREDS')], axis=1)
    #    domain = domain.drop(['INBREDS'], axis=1)
    #elif 'Maize_Line' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['Maize_Line'], prefix='Maize_Line')], axis=1)
    #    domain = domain.drop(['Maize_Line'], axis=1) 
    df = pd.concat([otu, domain], axis=1, sort=True, join='outer')
    #print(df.head())
    #data_microbioma = df[otu.columns].to_numpy(dtype=np.float32)
    #data_domain = df[domain.columns].to_numpy(dtype=np.float32)
    df_microbioma = df[otu.columns]
    df_domain = df[domain.columns]
    df_domain.head()
    df_microbioma_train, df_microbioma_no_train, df_domain_train, df_domain_no_train = \
        train_test_split(df_microbioma, df_domain, test_size=0.1, random_state=random_state)
    # Transfer learning subset
    df_microbioma_test, df_microbioma_transfer_learning, df_domain_test, df_domain_transfer_learning = \
        train_test_split(df_microbioma_no_train, df_domain_no_train, test_size=0.1, random_state=random_state)
    df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test = \
        train_test_split(df_microbioma_transfer_learning, df_domain_transfer_learning, test_size=0.3, random_state=random_state)
    
    return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns
    #return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns


def train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                batch_size, epochs, train_callbacks):
    all_models = model_fn()
    model, encoder_bioma, encoder_domain, decoder_bioma = all_models
    metrics_prefix = None
    if encoder_bioma is not None and encoder_domain is not None:
        x_train = (m_train, d_train)
        y_train = (m_train, m_train, z_train)
        x_test = (m_test, d_test)
        y_test = (m_test, m_test, z_test)
    elif encoder_bioma is not None:
        x_train = m_train
        y_train = m_train
        x_test = m_test
        y_test = m_test
        metrics_prefix = 'bioma'
    elif encoder_domain is not None:
        x_train = d_train
        y_train = m_train
        x_test = d_test
        y_test = m_test
        metrics_prefix = 'domain'

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(
        batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    r = model.fit(train_dataset,
                  epochs=epochs,
                  validation_data=val_dataset,
                  callbacks=train_callbacks,
                  verbose=0)
    if metrics_prefix is not None:
        old_keys = r.history
        r.history = {}
        for k, v in old_keys.items():
            if k == 'loss' or k == 'val_loss':
                new_key = k
            elif k.startswith('val_'):
                new_key = 'val_{}_{}'.format(metrics_prefix, k[4:])
            else:
                new_key = '{}_{}'.format(metrics_prefix, k)
            r.history[new_key] = v
    del val_dataset
    del train_dataset
    del x_train
    del y_train
    del x_test
    del y_test
    return r, all_models

def train_2(model_fn,
          data_microbioma,
          data_domain,
          latent_space=10,
          folds=5,
          epochs=20,
          batch_size=128,
          learning_rate_scheduler=ExpDecayScheluder(),
          random_seed=347,
          verbose=0):
    data_zeros_latent = np.zeros((data_microbioma.shape[0], latent_space), dtype=data_microbioma.dtype)
    results = []
    models = []
    train_callbacks = [
        callbacks.EarlyStopping(monitor='val_loss', patience=epochs + 1, restore_best_weights=True)]
    if verbose >= 0:
        train_callbacks += [TqdmCallback(verbose=verbose)]
    if learning_rate_scheduler is not None:
        train_callbacks += [learning_rate_scheduler.make()]

    if folds <= 1:
        m_train, m_test = data_microbioma, data_microbioma
        d_train, d_test = data_domain, data_domain
        z_train, z_test = data_zeros_latent, data_zeros_latent
        tf.random.set_seed(random_seed)
        r, m = train_kfold(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                           batch_size, epochs, train_callbacks)
        results.append(r)
        models.append(m)

    else: #EL PROBLEMA ESTA AQUI, QUE HACE FALTA UN 
        kf = KFold(n_splits=folds, random_state=random_seed, shuffle=True)
        tf.random.set_seed(random_seed)

        for train_index, test_index in kf.split(data_microbioma):
            m_train, m_test = data_microbioma[train_index], data_microbioma[test_index]
            #print(m_train)
            #d_train, d_test = data_domain[train_index], data_domain[test_index]
            if data_domain is None:
                d_train, d_test = None, None
            else:
                d_train, d_test = data_domain[train_index], data_domain[test_index]
            #print(d_train)
            #Esto de hacer el if else ha funcionado, pero no se si hace lo que debe bien
            z_train, z_test = data_zeros_latent[train_index], data_zeros_latent[test_index]
            r, m = train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                               batch_size, epochs, train_callbacks)
            results.append(r)
            models.append(m)
    return results, models

def perform_experiment_2_mod(cv_folds, epochs, batch_size, learning_rate, optimizer,
                       learning_rate_scheduler, input_transform, output_transform,
                       reconstruction_loss, latent_space, layers,
                       activation, activation_latent,
                       data_microbioma_train, data_domain_train,
                       show_results=True, device='/CPU:0'): #Show results cambiado de False  aTrue
    if input_transform is not None:
        input_transform = input_transform()
    #----------    
    if output_transform is not None:
        output_transform = output_transform()
    #----------      
    if reconstruction_loss.__class__.__name__ == 'MakeLoss':
        reconstruction_loss = reconstruction_loss.make()
    else:
        reconstruction_loss = reconstruction_loss()
    domain_layers = [l // 16 for l in layers] ####que es esto???? Esto es para las capas del domain
    #print(domain_layers)
    bioma_autoencoder = " -> ".join(["b"] +
                                    [str(l) for l in layers] +
                                    [str(latent_space)] +
                                    [str(l) for l in reversed(layers)] +
                                    ["b"])
    #---------- 
    #esto solo se utiliza para el texto, es irrelevante para nuestro error
    if data_domain_train is not None:
        domain_autoencoder = " -> ".join(["d"] +
                                     [str(l) for l in domain_layers] +
                                     [str(latent_space)] +
                                     [str(l) for l in reversed(layers)] +
                                     ["b"])
        
    else: 
        domain_autoencoder = " "
    #---------- 
    #donde se usa domain autoencoder?
    in_transform_name = input_transform.__class__.__name__ if input_transform else "none"
    out_transform_name = output_transform.__class__.__name__ if output_transform else "none"
    lr_scheduler_text = learning_rate_scheduler[
        1] if learning_rate_scheduler is not None else "none"
    lr_text = learning_rate if learning_rate_scheduler is not None else "constant = {}".format(
        learning_rate)
    learning_rate_scheduler = learning_rate_scheduler[
        0] if learning_rate_scheduler is not None else None
    optimizer = optimizer(learning_rate=learning_rate)
    #---------- 
    experiment_parameters = [
        ("Input transform", in_transform_name),
        ("Output transform", out_transform_name),
        ("Reconstruction Loss", reconstruction_loss.__class__.__name__),
        ("Latent Space", latent_space),
        ("Bioma Autoencoder", bioma_autoencoder),
        ("Domain Autoencoder", domain_autoencoder),
        ("Activation Encoder", activation),
        ("Activation Decoder", activation),
        ("Activation Latent", activation_latent),
        ("CV folds", cv_folds),
        ("Epochs", epochs),
        ("Batch Size", batch_size),
        ("Learning Rate Scheduler", lr_scheduler_text),
        ("Learning Rate", lr_text),
        ("Optimizer", optimizer.__class__.__name__),
    ]
    #----------  
    if show_results:
        md_text = ""
        md_text += "| Parameter             | Value         |\n"
        md_text += "|:----------------------|:--------------|\n"
        for n, v in experiment_parameters:
            md_text += "| {} | {} |\n".format(n, v)

        display(Markdown(md_text))
    #------------
    def create_model(print_data=False):
        bioma_shape=data_microbioma_train.shape[1]
        
        if data_domain_train is not None:
            domain_shape=data_domain_train.shape[1]
            #print("data_domain_train!=None")
        else:
            domain_shape=None
            #print("data_domain_train==None")
        models = autoencoder(bioma_shape=bioma_shape,
                             #bioma_shape=717,
                             domain_shape=domain_shape,
                             output_shape=bioma_shape,
                             #output_shape=717,
                             latent_space=latent_space,
                             bioma_layers=layers, #Esto es lo de [512,316]
                             domain_layers=domain_layers, #Esto son cada una de las layers divididas por 16
                             input_transform=input_transform,
                             output_transform=output_transform,
                             activation_function_encoder=activation,
                             activation_function_decoder=activation,
                             activation_function_latent=activation_latent)
        #Entiendo analizando lo demas que aqui NO esta el error
        #la funcion autoencoder esta en model.py (es la unica funcion en ese script)
        
        model, encoder_bioma, encoder_domain, decoder_bioma = models

        if print_data:
            plot_models(model, encoder_bioma, encoder_domain, decoder_bioma)
        compile_train(model,
                      encoder_bioma=encoder_bioma,
                      encoder_domain=encoder_domain,
                      reconstruction_error=reconstruction_loss,
                      encoded_comparison_error=losses.MeanAbsoluteError(),
                      metrics=get_experiment_metrics(input_transform, output_transform),
                      optimizer=optimizer)
        
        #print("He acabado create_model :)")
        return model, encoder_bioma, encoder_domain, decoder_bioma
    #-----------
    create_model(print_data=False)
    #-----------
    #Esta en esta seccion el problema, en train_2
    #print(data_domain_train)
    #print(latent_space)
    with tf.device(device):
        results, models = train_2(create_model,
                                data_microbioma_train,
                                data_domain_train,
                                latent_space=latent_space,
                                folds=cv_folds,
                                epochs=epochs,
                                batch_size=batch_size,
                                learning_rate_scheduler=learning_rate_scheduler,
                                verbose=-1)
    #----------
    validation_results = print_results(results, show_results=show_results)
    if show_results:
        display(Markdown("*************"))

    return experiment_parameters + validation_results, models, results

__params:__
 - "activat_func":["softmax","sigmoid","relu","tanh"]
 - "activ_ouput":["softmax","sigmoid","relu","tanh"]
 - "learning_rate":[0.01,0.001]

In [2]:
#Cargamos los datos
#nombres_metadatos = ["KCAL","PROT","TFAT","CARB","MOIS","ALC","CAFF","THEO","SUGR","FIBE","CALC","IRON","MAGN","PHOS","POTA","SODI","ZINC","COPP","SELE","VC","VB1","VB2","NIAC","VB6","FOLA","VB12","VARA","RET","BCAR","ACAR","CRYP","LYCO","LZ","ATOC","VK","CHOLE","SFAT","MFAT","PFAT","VITD","CHOLN"]
nombres_metadatos = ["PROT","TFAT","CARB","MOIS","ALC","CAFF","THEO","CALC","MAGN","POTA","ZINC","VC","VB1","VB6","VARA","ACAR","CRYP","LYCO","ATOC","VK","CHOLE","VITD"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='resultados_ana/datos_otus_filtrados/otu_table_especies_80.csv',metadata_filename='resultados_ana/metadatos_nutrientes_filtrados_22.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

#Preparamos las combinaciones pertinentes (5 mejores)
#####LA he liado aqui, deberia ser otra, hay que volver a hacerlo
combinations = [[100,64,0.001,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.001,optimizers.Adam,15,[512,256],"tanh","relu"],\ #este deberia ser 96
                [100,96,0.001,optimizers.Adam,15,[512,256,128],"tanh","tanh"],\ #deberia ser tanh, relu
                [100,96,0.01,optimizers.Adam,15,[512,256],"relu","tanh"],\
                [100,64,0.01,optimizers.Adam,10,[512,256],"relu","tanh"]]

### 0.001, tanh, tanh

In [3]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation=None)(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4666104316711426 | 0.4666104316711426 | 0.4666104316711426 |
| pearson_corr | 0.6399815082550049 | 0.6399815082550049 | 0.6399815082550049 |
| jensen_shannon_divergence | 1.2309167385101318 | 1.2309167385101318 | 1.2309167385101318 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4712810814380646 | 0.4712810814380646 | 0.4712810814380646 |
| pearson_corr | 0.6133289933204651 | 0.6133289933204651 | 0.6133289933204651 |
| jensen_shannon_divergence | 1.3000926971435547 | 1.3000926971435547 | 1.3000926971435547 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49360331892967224 | 0.49360331892967224 | 0.49360331892967224 |
| pearson_corr | 0.6167614459991455 | 0.6167614459991455 | 0.6167614459991455 |
| jensen_shannon_divergence | 1.3982597589492798 | 1.3982597589492798 | 1.3982597589492798 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46746793389320374 | 0.46746793389320374 | 0.46746793389320374 |
| pearson_corr | 0.6413127183914185 | 0.6413127183914185 | 0.6413127183914185 |
| jensen_shannon_divergence | 1.302670955657959 | 1.302670955657959 | 1.302670955657959 |


----------------------------



### 0.001, tanh, softmax

In [4]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.586988627910614 | 0.586988627910614 | 0.586988627910614 |
| pearson_corr | 0.5942000150680542 | 0.5942000150680542 | 0.5942000150680542 |
| jensen_shannon_divergence | 1.5406224727630615 | 1.5406224727630615 | 1.5406224727630615 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6788764595985413 | 0.6788764595985413 | 0.6788764595985413 |
| pearson_corr | 0.40872642397880554 | 0.40872642397880554 | 0.40872642397880554 |
| jensen_shannon_divergence | 2.1576194763183594 | 2.1576194763183594 | 2.1576194763183594 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5924661755561829 | 0.5924661755561829 | 0.5924661755561829 |
| pearson_corr | 0.4818921685218811 | 0.4818921685218811 | 0.4818921685218811 |
| jensen_shannon_divergence | 1.7611680030822754 | 1.7611680030822754 | 1.7611680030822754 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49580317735671997 | 0.49580317735671997 | 0.49580317735671997 |
| pearson_corr | 0.6158543229103088 | 0.6158543229103088 | 0.6158543229103088 |
| jensen_shannon_divergence | 1.405089020729065 | 1.405089020729065 | 1.405089020729065 |


----------------------------



### 0.001, tanh, sigmoid

In [5]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.532667338848114 | 0.532667338848114 | 0.532667338848114 |
| pearson_corr | 0.5948635935783386 | 0.5948635935783386 | 0.5948635935783386 |
| jensen_shannon_divergence | 1.3232924938201904 | 1.3232924938201904 | 1.3232924938201904 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4926283359527588 | 0.4926283359527588 | 0.4926283359527588 |
| pearson_corr | 0.597537636756897 | 0.597537636756897 | 0.597537636756897 |
| jensen_shannon_divergence | 1.2805906534194946 | 1.2805906534194946 | 1.2805906534194946 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5824371576309204 | 0.5824371576309204 | 0.5824371576309204 |
| pearson_corr | 0.5245718955993652 | 0.5245718955993652 | 0.5245718955993652 |
| jensen_shannon_divergence | 1.666982889175415 | 1.666982889175415 | 1.666982889175415 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47629714012145996 | 0.47629714012145996 | 0.47629714012145996 |
| pearson_corr | 0.6428070664405823 | 0.6428070664405823 | 0.6428070664405823 |
| jensen_shannon_divergence | 1.3273110389709473 | 1.3273110389709473 | 1.3273110389709473 |


----------------------------



### 0.001, tanh, relu

In [6]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5334022045135498 | 0.5334022045135498 | 0.5334022045135498 |
| pearson_corr | 0.5845524668693542 | 0.5845524668693542 | 0.5845524668693542 |
| jensen_shannon_divergence | 1.3018138408660889 | 1.3018138408660889 | 1.3018138408660889 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.480394572019577 | 0.480394572019577 | 0.480394572019577 |
| pearson_corr | 0.6137431263923645 | 0.6137431263923645 | 0.6137431263923645 |
| jensen_shannon_divergence | 1.311356544494629 | 1.311356544494629 | 1.311356544494629 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5763248801231384 | 0.5763248801231384 | 0.5763248801231384 |
| pearson_corr | 0.5414748787879944 | 0.5414748787879944 | 0.5414748787879944 |
| jensen_shannon_divergence | 1.6825300455093384 | 1.6825300455093384 | 1.6825300455093384 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4770861566066742 | 0.4770861566066742 | 0.4770861566066742 |
| pearson_corr | 0.6317811012268066 | 0.6317811012268066 | 0.6317811012268066 |
| jensen_shannon_divergence | 1.3847616910934448 | 1.3847616910934448 | 1.3847616910934448 |


----------------------------



### 0.001, softmax, tanh

In [7]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
  
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4749259054660797 | 0.4749259054660797 | 0.4749259054660797 |
| pearson_corr | 0.6169785857200623 | 0.6169785857200623 | 0.6169785857200623 |
| jensen_shannon_divergence | 1.2010539770126343 | 1.2010539770126343 | 1.2010539770126343 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4860888421535492 | 0.4860888421535492 | 0.4860888421535492 |
| pearson_corr | 0.6321198344230652 | 0.6321198344230652 | 0.6321198344230652 |
| jensen_shannon_divergence | 1.2382569313049316 | 1.2382569313049316 | 1.2382569313049316 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5568448901176453 | 0.5568448901176453 | 0.5568448901176453 |
| pearson_corr | 0.5595479607582092 | 0.5595479607582092 | 0.5595479607582092 |
| jensen_shannon_divergence | 1.6037875413894653 | 1.6037875413894653 | 1.6037875413894653 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5011512041091919 | 0.5011512041091919 | 0.5011512041091919 |
| pearson_corr | 0.594170868396759 | 0.594170868396759 | 0.594170868396759 |
| jensen_shannon_divergence | 1.4357837438583374 | 1.4357837438583374 | 1.4357837438583374 |


----------------------------



### 0.001, softmax, softmax

In [8]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5873225331306458 | 0.5873225331306458 | 0.5873225331306458 |
| pearson_corr | 0.5695122480392456 | 0.5695122480392456 | 0.5695122480392456 |
| jensen_shannon_divergence | 1.5207350254058838 | 1.5207350254058838 | 1.5207350254058838 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6751806139945984 | 0.6751806139945984 | 0.6751806139945984 |
| pearson_corr | 0.40019071102142334 | 0.40019071102142334 | 0.40019071102142334 |
| jensen_shannon_divergence | 2.1267290115356445 | 2.1267290115356445 | 2.1267290115356445 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5904536843299866 | 0.5904536843299866 | 0.5904536843299866 |
| pearson_corr | 0.4813489615917206 | 0.4813489615917206 | 0.4813489615917206 |
| jensen_shannon_divergence | 1.7280718088150024 | 1.7280718088150024 | 1.7280718088150024 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4968138337135315 | 0.4968138337135315 | 0.4968138337135315 |
| pearson_corr | 0.6168971061706543 | 0.6168971061706543 | 0.6168971061706543 |
| jensen_shannon_divergence | 1.4220815896987915 | 1.4220815896987915 | 1.4220815896987915 |


----------------------------



### 0.001, softmax, sigmoid

In [9]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5424084663391113 | 0.5424084663391113 | 0.5424084663391113 |
| pearson_corr | 0.5868552327156067 | 0.5868552327156067 | 0.5868552327156067 |
| jensen_shannon_divergence | 1.3533568382263184 | 1.3533568382263184 | 1.3533568382263184 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4866507947444916 | 0.4866507947444916 | 0.4866507947444916 |
| pearson_corr | 0.6318418383598328 | 0.6318418383598328 | 0.6318418383598328 |
| jensen_shannon_divergence | 1.238396167755127 | 1.238396167755127 | 1.238396167755127 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5908083319664001 | 0.5908083319664001 | 0.5908083319664001 |
| pearson_corr | 0.4991909861564636 | 0.4991909861564636 | 0.4991909861564636 |
| jensen_shannon_divergence | 1.6940926313400269 | 1.6940926313400269 | 1.6940926313400269 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4955788552761078 | 0.4955788552761078 | 0.4955788552761078 |
| pearson_corr | 0.617987871170044 | 0.617987871170044 | 0.617987871170044 |
| jensen_shannon_divergence | 1.420027256011963 | 1.420027256011963 | 1.420027256011963 |


----------------------------



### 0.001, softmax, relu

In [10]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')


    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5407905578613281 | 0.5407905578613281 | 0.5407905578613281 |
| pearson_corr | 0.5861202478408813 | 0.5861202478408813 | 0.5861202478408813 |
| jensen_shannon_divergence | 1.334692358970642 | 1.334692358970642 | 1.334692358970642 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5058145523071289 | 0.5058145523071289 | 0.5058145523071289 |
| pearson_corr | 0.5995340347290039 | 0.5995340347290039 | 0.5995340347290039 |
| jensen_shannon_divergence | 1.4027490615844727 | 1.4027490615844727 | 1.4027490615844727 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5876970291137695 | 0.5876970291137695 | 0.5876970291137695 |
| pearson_corr | 0.5043875575065613 | 0.5043875575065613 | 0.5043875575065613 |
| jensen_shannon_divergence | 1.684457778930664 | 1.684457778930664 | 1.684457778930664 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5161761045455933 | 0.5161761045455933 | 0.5161761045455933 |
| pearson_corr | 0.5866919755935669 | 0.5866919755935669 | 0.5866919755935669 |
| jensen_shannon_divergence | 1.5428745746612549 | 1.5428745746612549 | 1.5428745746612549 |


----------------------------



### 0.001, sigmoid, tanh

In [11]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.45292574167251587 | 0.45292574167251587 | 0.45292574167251587 |
| pearson_corr | 0.6506378650665283 | 0.6506378650665283 | 0.6506378650665283 |
| jensen_shannon_divergence | 1.1149582862854004 | 1.1149582862854004 | 1.1149582862854004 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47075629234313965 | 0.47075629234313965 | 0.47075629234313965 |
| pearson_corr | 0.6524497270584106 | 0.6524497270584106 | 0.6524497270584106 |
| jensen_shannon_divergence | 1.1872199773788452 | 1.1872199773788452 | 1.1872199773788452 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5234004855155945 | 0.5234004855155945 | 0.5234004855155945 |
| pearson_corr | 0.6056731343269348 | 0.6056731343269348 | 0.6056731343269348 |
| jensen_shannon_divergence | 1.4622228145599365 | 1.4622228145599365 | 1.4622228145599365 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4900318384170532 | 0.4900318384170532 | 0.4900318384170532 |
| pearson_corr | 0.6163731217384338 | 0.6163731217384338 | 0.6163731217384338 |
| jensen_shannon_divergence | 1.4021260738372803 | 1.4021260738372803 | 1.4021260738372803 |


----------------------------



### 0.001, sigmoid, softmax

In [12]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5852108597755432 | 0.5852108597755432 | 0.5852108597755432 |
| pearson_corr | 0.5793415307998657 | 0.5793415307998657 | 0.5793415307998657 |
| jensen_shannon_divergence | 1.5249993801116943 | 1.5249993801116943 | 1.5249993801116943 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6738884449005127 | 0.6738884449005127 | 0.6738884449005127 |
| pearson_corr | 0.4024542570114136 | 0.4024542570114136 | 0.4024542570114136 |
| jensen_shannon_divergence | 2.1107704639434814 | 2.1107704639434814 | 2.1107704639434814 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5901432037353516 | 0.5901432037353516 | 0.5901432037353516 |
| pearson_corr | 0.4834054708480835 | 0.4834054708480835 | 0.4834054708480835 |
| jensen_shannon_divergence | 1.7236802577972412 | 1.7236802577972412 | 1.7236802577972412 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4955628216266632 | 0.4955628216266632 | 0.4955628216266632 |
| pearson_corr | 0.6150037050247192 | 0.6150037050247192 | 0.6150037050247192 |
| jensen_shannon_divergence | 1.4202455282211304 | 1.4202455282211304 | 1.4202455282211304 |


----------------------------



### 0.001, sigmoid, sigmoid

In [13]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5408992767333984 | 0.5408992767333984 | 0.5408992767333984 |
| pearson_corr | 0.5922964215278625 | 0.5922964215278625 | 0.5922964215278625 |
| jensen_shannon_divergence | 1.346030592918396 | 1.346030592918396 | 1.346030592918396 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47584211826324463 | 0.47584211826324463 | 0.47584211826324463 |
| pearson_corr | 0.6505153179168701 | 0.6505153179168701 | 0.6505153179168701 |
| jensen_shannon_divergence | 1.1999902725219727 | 1.1999902725219727 | 1.1999902725219727 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5932461619377136 | 0.5932461619377136 | 0.5932461619377136 |
| pearson_corr | 0.5024570822715759 | 0.5024570822715759 | 0.5024570822715759 |
| jensen_shannon_divergence | 1.693668246269226 | 1.693668246269226 | 1.693668246269226 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49753570556640625 | 0.49753570556640625 | 0.49753570556640625 |
| pearson_corr | 0.6138851642608643 | 0.6138851642608643 | 0.6138851642608643 |
| jensen_shannon_divergence | 1.426950454711914 | 1.426950454711914 | 1.426950454711914 |


----------------------------



### 0.001, sigmoid, relu

In [14]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5329556465148926 | 0.5329556465148926 | 0.5329556465148926 |
| pearson_corr | 0.6057089567184448 | 0.6057089567184448 | 0.6057089567184448 |
| jensen_shannon_divergence | 1.2949601411819458 | 1.2949601411819458 | 1.2949601411819458 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4786314070224762 | 0.4786314070224762 | 0.4786314070224762 |
| pearson_corr | 0.6247378587722778 | 0.6247378587722778 | 0.6247378587722778 |
| jensen_shannon_divergence | 1.3116462230682373 | 1.3116462230682373 | 1.3116462230682373 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.588212788105011 | 0.588212788105011 | 0.588212788105011 |
| pearson_corr | 0.5013874769210815 | 0.5013874769210815 | 0.5013874769210815 |
| jensen_shannon_divergence | 1.7104216814041138 | 1.7104216814041138 | 1.7104216814041138 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5091392397880554 | 0.5091392397880554 | 0.5091392397880554 |
| pearson_corr | 0.5934340953826904 | 0.5934340953826904 | 0.5934340953826904 |
| jensen_shannon_divergence | 1.5164493322372437 | 1.5164493322372437 | 1.5164493322372437 |


----------------------------



### 0.001, relu, tanh

In [15]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6219472885131836 | 0.6219472885131836 | 0.6219472885131836 |
| pearson_corr | 0.5768758654594421 | 0.5768758654594421 | 0.5768758654594421 |
| jensen_shannon_divergence | 2.4853711128234863 | 2.4853711128234863 | 2.4853711128234863 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6728084087371826 | 0.6728084087371826 | 0.6728084087371826 |
| pearson_corr | 0.40082213282585144 | 0.40082213282585144 | 0.40082213282585144 |
| jensen_shannon_divergence | 2.2855660915374756 | 2.2855660915374756 | 2.2855660915374756 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7280701398849487 | 0.7280701398849487 | 0.7280701398849487 |
| pearson_corr | 0.37260672450065613 | 0.37260672450065613 | 0.37260672450065613 |
| jensen_shannon_divergence | 3.200467586517334 | 3.200467586517334 | 3.200467586517334 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6892411112785339 | 0.6892411112785339 | 0.6892411112785339 |
| pearson_corr | 0.29762476682662964 | 0.29762476682662964 | 0.29762476682662964 |
| jensen_shannon_divergence | 3.4883010387420654 | 3.4883010387420654 | 3.4883010387420654 |


----------------------------



### 0.001, relu, softmax

In [16]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.722378671169281 | 0.722378671169281 | 0.722378671169281 |
| pearson_corr | 0.3621353805065155 | 0.3621353805065155 | 0.3621353805065155 |
| jensen_shannon_divergence | 2.727346181869507 | 2.727346181869507 | 2.727346181869507 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8037365674972534 | 0.8037365674972534 | 0.8037365674972534 |
| pearson_corr | 0.12278395146131516 | 0.12278395146131516 | 0.12278395146131516 |
| jensen_shannon_divergence | 3.5906176567077637 | 3.5906176567077637 | 3.5906176567077637 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7197839617729187 | 0.7197839617729187 | 0.7197839617729187 |
| pearson_corr | 0.18534798920154572 | 0.18534798920154572 | 0.18534798920154572 |
| jensen_shannon_divergence | 2.555655002593994 | 2.555655002593994 | 2.555655002593994 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5335311889648438 | 0.5335311889648438 | 0.5335311889648438 |
| pearson_corr | 0.5180814862251282 | 0.5180814862251282 | 0.5180814862251282 |
| jensen_shannon_divergence | 1.6139085292816162 | 1.6139085292816162 | 1.6139085292816162 |


----------------------------



### 0.001, relu, sigmoid

In [17]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6116445660591125 | 0.6116445660591125 | 0.6116445660591125 |
| pearson_corr | 0.423698753118515 | 0.423698753118515 | 0.423698753118515 |
| jensen_shannon_divergence | 1.85905921459198 | 1.85905921459198 | 1.85905921459198 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5753889083862305 | 0.5753889083862305 | 0.5753889083862305 |
| pearson_corr | 0.4730903208255768 | 0.4730903208255768 | 0.4730903208255768 |
| jensen_shannon_divergence | 1.5888479948043823 | 1.5888479948043823 | 1.5888479948043823 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6016227006912231 | 0.6016227006912231 | 0.6016227006912231 |
| pearson_corr | 0.3993355333805084 | 0.3993355333805084 | 0.3993355333805084 |
| jensen_shannon_divergence | 1.6724246740341187 | 1.6724246740341187 | 1.6724246740341187 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5282157063484192 | 0.5282157063484192 | 0.5282157063484192 |
| pearson_corr | 0.5559995174407959 | 0.5559995174407959 | 0.5559995174407959 |
| jensen_shannon_divergence | 1.5815693140029907 | 1.5815693140029907 | 1.5815693140029907 |


----------------------------



### 0.001, relu, relu

In [18]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7541691660881042 | 0.7541691660881042 | 0.7541691660881042 |
| pearson_corr | 0.39303353428840637 | 0.39303353428840637 | 0.39303353428840637 |
| jensen_shannon_divergence | 2.7582242488861084 | 2.7582242488861084 | 2.7582242488861084 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7887640595436096 | 0.7887640595436096 | 0.7887640595436096 |
| pearson_corr | 0.20393438637256622 | 0.20393438637256622 | 0.20393438637256622 |
| jensen_shannon_divergence | 3.34212327003479 | 3.34212327003479 | 3.34212327003479 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6499668955802917 | 0.6499668955802917 | 0.6499668955802917 |
| pearson_corr | 0.34821632504463196 | 0.34821632504463196 | 0.34821632504463196 |
| jensen_shannon_divergence | 2.0826337337493896 | 2.0826337337493896 | 2.0826337337493896 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5357425212860107 | 0.5357425212860107 | 0.5357425212860107 |
| pearson_corr | 0.5860261917114258 | 0.5860261917114258 | 0.5860261917114258 |
| jensen_shannon_divergence | 1.6744686365127563 | 1.6744686365127563 | 1.6744686365127563 |


----------------------------



---------------------------
### 0.01, tanh, tanh

In [19]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.474091112613678 | 0.474091112613678 | 0.474091112613678 |
| pearson_corr | 0.6171327233314514 | 0.6171327233314514 | 0.6171327233314514 |
| jensen_shannon_divergence | 1.219081997871399 | 1.219081997871399 | 1.219081997871399 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4869934320449829 | 0.4869934320449829 | 0.4869934320449829 |
| pearson_corr | 0.6280184388160706 | 0.6280184388160706 | 0.6280184388160706 |
| jensen_shannon_divergence | 1.2627395391464233 | 1.2627395391464233 | 1.2627395391464233 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.568156898021698 | 0.568156898021698 | 0.568156898021698 |
| pearson_corr | 0.5312444567680359 | 0.5312444567680359 | 0.5312444567680359 |
| jensen_shannon_divergence | 1.6486533880233765 | 1.6486533880233765 | 1.6486533880233765 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49966999888420105 | 0.49966999888420105 | 0.49966999888420105 |
| pearson_corr | 0.5936757922172546 | 0.5936757922172546 | 0.5936757922172546 |
| jensen_shannon_divergence | 1.4305322170257568 | 1.4305322170257568 | 1.4305322170257568 |


----------------------------



### 0.01, tanh, softmax

In [20]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5890068411827087 | 0.5890068411827087 | 0.5890068411827087 |
| pearson_corr | 0.5695511102676392 | 0.5695511102676392 | 0.5695511102676392 |
| jensen_shannon_divergence | 1.5309932231903076 | 1.5309932231903076 | 1.5309932231903076 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6717161536216736 | 0.6717161536216736 | 0.6717161536216736 |
| pearson_corr | 0.40583059191703796 | 0.40583059191703796 | 0.40583059191703796 |
| jensen_shannon_divergence | 2.106616497039795 | 2.106616497039795 | 2.106616497039795 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5912229418754578 | 0.5912229418754578 | 0.5912229418754578 |
| pearson_corr | 0.481029212474823 | 0.481029212474823 | 0.481029212474823 |
| jensen_shannon_divergence | 1.7280992269515991 | 1.7280992269515991 | 1.7280992269515991 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.49611330032348633 | 0.49611330032348633 | 0.49611330032348633 |
| pearson_corr | 0.6183952689170837 | 0.6183952689170837 | 0.6183952689170837 |
| jensen_shannon_divergence | 1.418558120727539 | 1.418558120727539 | 1.418558120727539 |


----------------------------



### 0.01, tanh, sigmoid

In [21]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5424045324325562 | 0.5424045324325562 | 0.5424045324325562 |
| pearson_corr | 0.5869810581207275 | 0.5869810581207275 | 0.5869810581207275 |
| jensen_shannon_divergence | 1.3508694171905518 | 1.3508694171905518 | 1.3508694171905518 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47769632935523987 | 0.47769632935523987 | 0.47769632935523987 |
| pearson_corr | 0.6396298408508301 | 0.6396298408508301 | 0.6396298408508301 |
| jensen_shannon_divergence | 1.2200433015823364 | 1.2200433015823364 | 1.2200433015823364 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5933905839920044 | 0.5933905839920044 | 0.5933905839920044 |
| pearson_corr | 0.4882001578807831 | 0.4882001578807831 | 0.4882001578807831 |
| jensen_shannon_divergence | 1.7133604288101196 | 1.7133604288101196 | 1.7133604288101196 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4963834583759308 | 0.4963834583759308 | 0.4963834583759308 |
| pearson_corr | 0.6168468594551086 | 0.6168468594551086 | 0.6168468594551086 |
| jensen_shannon_divergence | 1.4211859703063965 | 1.4211859703063965 | 1.4211859703063965 |


----------------------------



### 0.01, tanh, relu

In [22]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5442017912864685 | 0.5442017912864685 | 0.5442017912864685 |
| pearson_corr | 0.5801864266395569 | 0.5801864266395569 | 0.5801864266395569 |
| jensen_shannon_divergence | 1.3442027568817139 | 1.3442027568817139 | 1.3442027568817139 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47905945777893066 | 0.47905945777893066 | 0.47905945777893066 |
| pearson_corr | 0.6278253793716431 | 0.6278253793716431 | 0.6278253793716431 |
| jensen_shannon_divergence | 1.28011155128479 | 1.28011155128479 | 1.28011155128479 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6472703218460083 | 0.6472703218460083 | 0.6472703218460083 |
| pearson_corr | 0.3543688952922821 | 0.3543688952922821 | 0.3543688952922821 |
| jensen_shannon_divergence | 2.0513923168182373 | 2.0513923168182373 | 2.0513923168182373 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4927811026573181 | 0.4927811026573181 | 0.4927811026573181 |
| pearson_corr | 0.6283842921257019 | 0.6283842921257019 | 0.6283842921257019 |
| jensen_shannon_divergence | 1.3787132501602173 | 1.3787132501602173 | 1.3787132501602173 |


----------------------------



### 0.01, softmax, tanh

In [23]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47329938411712646 | 0.47329938411712646 | 0.47329938411712646 |
| pearson_corr | 0.6189389824867249 | 0.6189389824867249 | 0.6189389824867249 |
| jensen_shannon_divergence | 1.1994370222091675 | 1.1994370222091675 | 1.1994370222091675 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48165759444236755 | 0.48165759444236755 | 0.48165759444236755 |
| pearson_corr | 0.6362542510032654 | 0.6362542510032654 | 0.6362542510032654 |
| jensen_shannon_divergence | 1.2282445430755615 | 1.2282445430755615 | 1.2282445430755615 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5574375987052917 | 0.5574375987052917 | 0.5574375987052917 |
| pearson_corr | 0.5553440451622009 | 0.5553440451622009 | 0.5553440451622009 |
| jensen_shannon_divergence | 1.606074571609497 | 1.606074571609497 | 1.606074571609497 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5017693638801575 | 0.5017693638801575 | 0.5017693638801575 |
| pearson_corr | 0.5955280661582947 | 0.5955280661582947 | 0.5955280661582947 |
| jensen_shannon_divergence | 1.4412851333618164 | 1.4412851333618164 | 1.4412851333618164 |


----------------------------



### 0.01, softmax, softmax

In [24]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5884733200073242 | 0.5884733200073242 | 0.5884733200073242 |
| pearson_corr | 0.5692443251609802 | 0.5692443251609802 | 0.5692443251609802 |
| jensen_shannon_divergence | 1.527906894683838 | 1.527906894683838 | 1.527906894683838 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6723020672798157 | 0.6723020672798157 | 0.6723020672798157 |
| pearson_corr | 0.4022471308708191 | 0.4022471308708191 | 0.4022471308708191 |
| jensen_shannon_divergence | 2.1093900203704834 | 2.1093900203704834 | 2.1093900203704834 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5913715958595276 | 0.5913715958595276 | 0.5913715958595276 |
| pearson_corr | 0.4817619025707245 | 0.4817619025707245 | 0.4817619025707245 |
| jensen_shannon_divergence | 1.7258583307266235 | 1.7258583307266235 | 1.7258583307266235 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4971248507499695 | 0.4971248507499695 | 0.4971248507499695 |
| pearson_corr | 0.6158589124679565 | 0.6158589124679565 | 0.6158589124679565 |
| jensen_shannon_divergence | 1.4244366884231567 | 1.4244366884231567 | 1.4244366884231567 |


----------------------------



### 0.01, softmax, sigmoid

In [25]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5413151979446411 | 0.5413151979446411 | 0.5413151979446411 |
| pearson_corr | 0.5919100642204285 | 0.5919100642204285 | 0.5919100642204285 |
| jensen_shannon_divergence | 1.3468931913375854 | 1.3468931913375854 | 1.3468931913375854 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4777947664260864 | 0.4777947664260864 | 0.4777947664260864 |
| pearson_corr | 0.6444529294967651 | 0.6444529294967651 | 0.6444529294967651 |
| jensen_shannon_divergence | 1.2101408243179321 | 1.2101408243179321 | 1.2101408243179321 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5935360789299011 | 0.5935360789299011 | 0.5935360789299011 |
| pearson_corr | 0.5019572973251343 | 0.5019572973251343 | 0.5019572973251343 |
| jensen_shannon_divergence | 1.6946349143981934 | 1.6946349143981934 | 1.6946349143981934 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4976891577243805 | 0.4976891577243805 | 0.4976891577243805 |
| pearson_corr | 0.6135696768760681 | 0.6135696768760681 | 0.6135696768760681 |
| jensen_shannon_divergence | 1.4278128147125244 | 1.4278128147125244 | 1.4278128147125244 |


----------------------------



### 0.01, softmax, relu

In [26]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.544954240322113 | 0.544954240322113 | 0.544954240322113 |
| pearson_corr | 0.5806062817573547 | 0.5806062817573547 | 0.5806062817573547 |
| jensen_shannon_divergence | 1.3496800661087036 | 1.3496800661087036 | 1.3496800661087036 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.488530695438385 | 0.488530695438385 | 0.488530695438385 |
| pearson_corr | 0.6209768056869507 | 0.6209768056869507 | 0.6209768056869507 |
| jensen_shannon_divergence | 1.3507972955703735 | 1.3507972955703735 | 1.3507972955703735 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5887959003448486 | 0.5887959003448486 | 0.5887959003448486 |
| pearson_corr | 0.49505120515823364 | 0.49505120515823364 | 0.49505120515823364 |
| jensen_shannon_divergence | 1.701473593711853 | 1.701473593711853 | 1.701473593711853 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5225422382354736 | 0.5225422382354736 | 0.5225422382354736 |
| pearson_corr | 0.5760942697525024 | 0.5760942697525024 | 0.5760942697525024 |
| jensen_shannon_divergence | 1.5785287618637085 | 1.5785287618637085 | 1.5785287618637085 |


----------------------------



### 0.01, sigmoid, tanh

In [27]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46412286162376404 | 0.46412286162376404 | 0.46412286162376404 |
| pearson_corr | 0.6297613978385925 | 0.6297613978385925 | 0.6297613978385925 |
| jensen_shannon_divergence | 1.167609691619873 | 1.167609691619873 | 1.167609691619873 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46452510356903076 | 0.46452510356903076 | 0.46452510356903076 |
| pearson_corr | 0.6662938594818115 | 0.6662938594818115 | 0.6662938594818115 |
| jensen_shannon_divergence | 1.16500985622406 | 1.16500985622406 | 1.16500985622406 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5488762259483337 | 0.5488762259483337 | 0.5488762259483337 |
| pearson_corr | 0.5711533427238464 | 0.5711533427238464 | 0.5711533427238464 |
| jensen_shannon_divergence | 1.563284158706665 | 1.563284158706665 | 1.563284158706665 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4942728281021118 | 0.4942728281021118 | 0.4942728281021118 |
| pearson_corr | 0.6068559885025024 | 0.6068559885025024 | 0.6068559885025024 |
| jensen_shannon_divergence | 1.4137170314788818 | 1.4137170314788818 | 1.4137170314788818 |


----------------------------



### 0.01, sigmoid, softmax

In [28]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5890363454818726 | 0.5890363454818726 | 0.5890363454818726 |
| pearson_corr | 0.5696003437042236 | 0.5696003437042236 | 0.5696003437042236 |
| jensen_shannon_divergence | 1.531173586845398 | 1.531173586845398 | 1.531173586845398 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6721305251121521 | 0.6721305251121521 | 0.6721305251121521 |
| pearson_corr | 0.40291932225227356 | 0.40291932225227356 | 0.40291932225227356 |
| jensen_shannon_divergence | 2.108981132507324 | 2.108981132507324 | 2.108981132507324 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5918797850608826 | 0.5918797850608826 | 0.5918797850608826 |
| pearson_corr | 0.4821167290210724 | 0.4821167290210724 | 0.4821167290210724 |
| jensen_shannon_divergence | 1.7253587245941162 | 1.7253587245941162 | 1.7253587245941162 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4950781762599945 | 0.4950781762599945 | 0.4950781762599945 |
| pearson_corr | 0.6190834641456604 | 0.6190834641456604 | 0.6190834641456604 |
| jensen_shannon_divergence | 1.413927435874939 | 1.413927435874939 | 1.413927435874939 |


----------------------------



### 0.01, sigmoid, sigmoid

In [29]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5428716540336609 | 0.5428716540336609 | 0.5428716540336609 |
| pearson_corr | 0.5864921808242798 | 0.5864921808242798 | 0.5864921808242798 |
| jensen_shannon_divergence | 1.3526524305343628 | 1.3526524305343628 | 1.3526524305343628 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4806676506996155 | 0.4806676506996155 | 0.4806676506996155 |
| pearson_corr | 0.637134850025177 | 0.637134850025177 | 0.637134850025177 |
| jensen_shannon_divergence | 1.2266535758972168 | 1.2266535758972168 | 1.2266535758972168 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5933076739311218 | 0.5933076739311218 | 0.5933076739311218 |
| pearson_corr | 0.4907061457633972 | 0.4907061457633972 | 0.4907061457633972 |
| jensen_shannon_divergence | 1.7087472677230835 | 1.7087472677230835 | 1.7087472677230835 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4973767399787903 | 0.4973767399787903 | 0.4973767399787903 |
| pearson_corr | 0.6145907640457153 | 0.6145907640457153 | 0.6145907640457153 |
| jensen_shannon_divergence | 1.4251785278320312 | 1.4251785278320312 | 1.4251785278320312 |


----------------------------



### 0.01, sigmoid, relu

In [30]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5464217662811279 | 0.5464217662811279 | 0.5464217662811279 |
| pearson_corr | 0.5759163498878479 | 0.5759163498878479 | 0.5759163498878479 |
| jensen_shannon_divergence | 1.3523391485214233 | 1.3523391485214233 | 1.3523391485214233 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48840978741645813 | 0.48840978741645813 | 0.48840978741645813 |
| pearson_corr | 0.6339693069458008 | 0.6339693069458008 | 0.6339693069458008 |
| jensen_shannon_divergence | 1.2972055673599243 | 1.2972055673599243 | 1.2972055673599243 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5863692760467529 | 0.5863692760467529 | 0.5863692760467529 |
| pearson_corr | 0.4665094017982483 | 0.4665094017982483 | 0.4665094017982483 |
| jensen_shannon_divergence | 1.8176243305206299 | 1.8176243305206299 | 1.8176243305206299 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4998079538345337 | 0.4998079538345337 | 0.4998079538345337 |
| pearson_corr | 0.6131545901298523 | 0.6131545901298523 | 0.6131545901298523 |
| jensen_shannon_divergence | 1.4269328117370605 | 1.4269328117370605 | 1.4269328117370605 |


----------------------------



### 0.01, relu, tanh

In [31]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5925171375274658 | 0.5925171375274658 | 0.5925171375274658 |
| pearson_corr | 0.5802437663078308 | 0.5802437663078308 | 0.5802437663078308 |
| jensen_shannon_divergence | 2.2235240936279297 | 2.2235240936279297 | 2.2235240936279297 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7096750736236572 | 0.7096750736236572 | 0.7096750736236572 |
| pearson_corr | 0.36807700991630554 | 0.36807700991630554 | 0.36807700991630554 |
| jensen_shannon_divergence | 2.7388877868652344 | 2.7388877868652344 | 2.7388877868652344 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.900239109992981 | 0.900239109992981 | 0.900239109992981 |
| pearson_corr | 0.11126795411109924 | 0.11126795411109924 | 0.11126795411109924 |
| jensen_shannon_divergence | 6.5262131690979 | 6.5262131690979 | 6.5262131690979 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7095433473587036 | 0.7095433473587036 | 0.7095433473587036 |
| pearson_corr | 0.21035796403884888 | 0.21035796403884888 | 0.21035796403884888 |
| jensen_shannon_divergence | 3.391899347305298 | 3.391899347305298 | 3.391899347305298 |


----------------------------



### 0.01, relu, softmax

In [32]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.722378671169281 | 0.722378671169281 | 0.722378671169281 |
| pearson_corr | 0.3621353805065155 | 0.3621353805065155 | 0.3621353805065155 |
| jensen_shannon_divergence | 2.727346181869507 | 2.727346181869507 | 2.727346181869507 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6920798420906067 | 0.6920798420906067 | 0.6920798420906067 |
| pearson_corr | 0.32933932542800903 | 0.32933932542800903 | 0.32933932542800903 |
| jensen_shannon_divergence | 2.3396284580230713 | 2.3396284580230713 | 2.3396284580230713 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5881101489067078 | 0.5881101489067078 | 0.5881101489067078 |
| pearson_corr | 0.45865598320961 | 0.45865598320961 | 0.45865598320961 |
| jensen_shannon_divergence | 1.8767889738082886 | 1.8767889738082886 | 1.8767889738082886 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5335311889648438 | 0.5335311889648438 | 0.5335311889648438 |
| pearson_corr | 0.5180814862251282 | 0.5180814862251282 | 0.5180814862251282 |
| jensen_shannon_divergence | 1.6139085292816162 | 1.6139085292816162 | 1.6139085292816162 |


----------------------------



### 0.01, relu, sigmoid

In [33]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6675030589103699 | 0.6675030589103699 | 0.6675030589103699 |
| pearson_corr | 0.3960495889186859 | 0.3960495889186859 | 0.3960495889186859 |
| jensen_shannon_divergence | 2.0206642150878906 | 2.0206642150878906 | 2.0206642150878906 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5753889083862305 | 0.5753889083862305 | 0.5753889083862305 |
| pearson_corr | 0.4730903208255768 | 0.4730903208255768 | 0.4730903208255768 |
| jensen_shannon_divergence | 1.5888479948043823 | 1.5888479948043823 | 1.5888479948043823 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6135459542274475 | 0.6135459542274475 | 0.6135459542274475 |
| pearson_corr | 0.34997785091400146 | 0.34997785091400146 | 0.34997785091400146 |
| jensen_shannon_divergence | 1.9693576097488403 | 1.9693576097488403 | 1.9693576097488403 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6403829455375671 | 0.6403829455375671 | 0.6403829455375671 |
| pearson_corr | 0.3469322621822357 | 0.3469322621822357 | 0.3469322621822357 |
| jensen_shannon_divergence | 1.9785798788070679 | 1.9785798788070679 | 1.9785798788070679 |


----------------------------



### 0.01, relu, relu

In [34]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7534201741218567 | 0.7534201741218567 | 0.7534201741218567 |
| pearson_corr | 0.39772671461105347 | 0.39772671461105347 | 0.39772671461105347 |
| jensen_shannon_divergence | 2.7222695350646973 | 2.7222695350646973 | 2.7222695350646973 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'relu']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7881259322166443 | 0.7881259322166443 | 0.7881259322166443 |
| pearson_corr | 0.22082915902137756 | 0.22082915902137756 | 0.22082915902137756 |
| jensen_shannon_divergence | 3.173801898956299 | 3.173801898956299 | 3.173801898956299 |


----------------------------

[100, 96, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6472703218460083 | 0.6472703218460083 | 0.6472703218460083 |
| pearson_corr | 0.3543688952922821 | 0.3543688952922821 | 0.3543688952922821 |
| jensen_shannon_divergence | 2.0513923168182373 | 2.0513923168182373 | 2.0513923168182373 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'relu', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5292101502418518 | 0.5292101502418518 | 0.5292101502418518 |
| pearson_corr | 0.5924198627471924 | 0.5924198627471924 | 0.5924198627471924 |
| jensen_shannon_divergence | 1.5824764966964722 | 1.5824764966964722 | 1.5824764966964722 |


----------------------------

